In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
import os
from PIL import Image,ImageOps
import random
import tensorflow as tf
import time
from datetime import datetime

In [3]:
TRAIN_PATH = './data/train/'
LOGS_Path = "./logs"
BATCH_SIZE = 8

In [4]:
files_list = glob.glob(os.path.join(TRAIN_PATH,"**/*"))
def get_img_batch(files_list,batch_size=32,size=(224,224),should_normalise=True,should_noise=True):
   
    batch_cover = []
    batch_secret = []


    for i in range(batch_size):
        img_secret_path = random.choice(files_list)
        img_cover_path = random.choice(files_list)
        
        img_secret = Image.open(img_secret_path).convert("RGB")
        img_cover = Image.open(img_cover_path).convert("RGB")

        img_secret = np.array(ImageOps.fit(img_secret,size),dtype=np.float32)
        img_cover = np.array(ImageOps.fit(img_cover,size),dtype=np.float32)
        
        img_secret /= 255.
        img_cover /= 255.
        
        
        
        batch_cover.append(img_cover)
        batch_secret.append(img_secret)
        
    batch_cover,batch_secret = np.array(batch_cover) , np.array(batch_secret)
    
    batch_cover = (batch_cover -  np.array([0.485, 0.456, 0.406]))/np.array([0.229, 0.224, 0.225])
    batch_secret = (batch_secret -  np.array([0.485, 0.456, 0.406]))/np.array([0.229, 0.224, 0.225])

    return batch_cover,batch_secret
    

In [5]:
def get_prep_network_op(secret_tensor):
    
    with tf.variable_scope('prep_net'):
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=3,padding='same',name="1",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4",activation=tf.nn.relu)
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=4,padding='same',name="1",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2",activation=tf.nn.relu)           
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4",activation=tf.nn.relu)

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=secret_tensor,filters=50,kernel_size=5,padding='same',name="1",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2",activation=tf.nn.relu)           
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4",activation=tf.nn.relu)
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5",activation=tf.nn.relu)
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4",activation=tf.nn.relu)
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3",activation=tf.nn.relu)
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')


        return concat_final

    
def get_hiding_network_op(cover_tensor,prep_output):
    
    with tf.variable_scope('hide_net'):
        concat_input = tf.concat([cover_tensor,prep_output],axis=3,name='images_features_concat')
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=3,padding='same',name="1",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4",activation=tf.nn.relu)
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=4,padding='same',name="1",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2",activation=tf.nn.relu)          
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4",activation=tf.nn.relu)

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=concat_input,filters=50,kernel_size=5,padding='same',name="1",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2",activation=tf.nn.relu)          
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4",activation=tf.nn.relu)
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5",activation=tf.nn.relu)
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4",activation=tf.nn.relu)
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3",activation=tf.nn.relu)
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')
    
        output = tf.layers.conv2d(inputs=concat_final,filters=3,kernel_size=1,padding='same',name='output')



        return output
    
        
        
def get_reveal_network_op(container_tensor):
    
    with tf.variable_scope('reveal_net'):
        
        with tf.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=3,padding='same',name="1",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="2",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="3",activation=tf.nn.relu)
            conv_3x3 = tf.layers.conv2d(inputs=conv_3x3,filters=50,kernel_size=3,padding='same',name="4",activation=tf.nn.relu)
            
        with tf.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=4,padding='same',name="1",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="2",activation=tf.nn.relu)          
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="3",activation=tf.nn.relu)
            conv_4x4 = tf.layers.conv2d(inputs=conv_4x4,filters=50,kernel_size=4,padding='same',name="4",activation=tf.nn.relu)

        with tf.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.layers.conv2d(inputs=container_tensor,filters=50,kernel_size=5,padding='same',name="1",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="2",activation=tf.nn.relu)           
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="3",activation=tf.nn.relu)
            conv_5x5 = tf.layers.conv2d(inputs=conv_5x5,filters=50,kernel_size=5,padding='same',name="4",activation=tf.nn.relu)
            
        concat_1 = tf.concat([conv_3x3,conv_4x4,conv_5x5],axis=3,name='concat_1')
        
        conv_5x5 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_5x5",activation=tf.nn.relu)
        conv_4x4 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_4x4",activation=tf.nn.relu)
        conv_3x3 = tf.layers.conv2d(inputs=concat_1,filters=50,kernel_size=5,padding='same',name="final_3x3",activation=tf.nn.relu)
        
        concat_final = tf.concat([conv_5x5,conv_4x4,conv_3x3],axis=3,name='concat_final')
    
    output = tf.layers.conv2d(inputs=concat_final,filters=3,kernel_size=1,padding='same',name='output')

    return output


def get_loss_op(secret_true,secret_pred,cover_true,cover_pred,beta=.5):
    
    with tf.variable_scope("losses"):
        beta = tf.constant(beta,name="beta")
        secret_mse = tf.losses.mean_squared_error(secret_true,secret_pred)
        cover_mse = tf.losses.mean_squared_error(cover_true,cover_pred)
        final_loss = cover_mse + beta*secret_mse
        return final_loss , secret_mse , cover_mse 

def get_noise_layer_op(tensor,std=.1):
    with tf.variable_scope("noise_layer"):
        return tensor + tf.random_normal(shape=tf.shape(tensor), mean=0.0, stddev=std, dtype=tf.float32) 

In [6]:
def tensor_to_img_op(tensor):
    with tf.variable_scope("",reuse=True):
        t = tensor*tf.convert_to_tensor([0.229, 0.224, 0.225]) + tf.convert_to_tensor([0.485, 0.456, 0.406])
        return tf.clip_by_value(t,0,1)

In [7]:
def prepare_training_graph():
    secret_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_prep")
    cover_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_hide")
    
    prep_output_op = get_prep_network_op(secret_tensor)
    hiding_output_op = get_hiding_network_op(cover_tensor=cover_tensor,prep_output=prep_output_op)
    noise_add_op = get_noise_layer_op(hiding_output_op)
    reveal_output_op = get_reveal_network_op(noise_add_op)
    
    loss_op,secret_loss_op,cover_loss_op = get_loss_op(secret_tensor,reveal_output_op,cover_tensor,hiding_output_op)

    minimize_op = tf.train.AdamOptimizer(0.0001).minimize(loss_op)
    
    tf.summary.scalar('loss', loss_op)
    tf.summary.scalar('reveal_network_loss', secret_loss_op)
    tf.summary.scalar('cover_network_loss', cover_loss_op)


    tf.summary.image('secret',tensor_to_img_op(secret_tensor),max_outputs=1)
    tf.summary.image('cover',tensor_to_img_op(cover_tensor),max_outputs=1)
    tf.summary.image('hidden',tensor_to_img_op(hiding_output_op),max_outputs=1)
    tf.summary.image('hidden_noisy',tensor_to_img_op(noise_add_op),max_outputs=1)
    tf.summary.image('revealed',tensor_to_img_op(reveal_output_op),max_outputs=1)

    merged_summary_op = tf.summary.merge_all()
    
    return minimize_op, merged_summary_op 


    
    

In [8]:
sess = tf.InteractiveSession(graph=tf.Graph())
train_op , summary_op = prepare_training_graph()

In [9]:
writer = tf.summary.FileWriter(f"logs/run1",sess.graph)
saver = tf.train.Saver()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
total_steps = len(files_list)//BATCH_SIZE + 1
for ep in range(3000):
    
    for step in range(total_steps):
        covers,secrets = get_img_batch(files_list=files_list,batch_size=BATCH_SIZE)
        sess.run([train_op],feed_dict={"input_prep:0":secrets,"input_hide:0":covers})
        
        if step % 10 ==0 :
            summary = sess.run(summary_op,feed_dict={"input_prep:0":secrets,"input_hide:0":covers})
            writer.add_summary(summary,ep*total_steps + step)
            
    
    save_path = saver.save(sess, f"./checkpoints/run1-step_{ep*total_steps + step}.chpk")

In [ ]:
# secret_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_prep")
# cover_tensor = tf.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="input_hide")


# prep_output_op = prep_network(secret_tensor)

# hiding_output_op = hiding_network(cover_tensor=cover_tensor,prep_output=prep_output_op)

# noise_add_op = get_noise_op(hiding_output_op)

# reveal_output_op = reveal_network(noise_add_op)



# loss_op = get_loss_op(secret_tensor,reveal_output_op,cover_tensor,hiding_output_op)
# minimize_op = tf.train.AdamOptimizer(0.0001,epsilon=1e-06).minimize(loss_op)
    

# def tensor_to_img_op(tensor):
#     with tf.variable_scope("",reuse=True):
#         t = tensor*tf.convert_to_tensor([0.229, 0.224, 0.225]) + tf.convert_to_tensor([0.485, 0.456, 0.406])
#         return tf.clip_by_value(t,0,1)


# tf.summary.scalar('loss', loss_op)
# tf.summary.image('secret',tensor_to_img_op(secret_tensor),max_outputs=1)
# tf.summary.image('cover',tensor_to_img_op(cover_tensor),max_outputs=1)
# tf.summary.image('hidden',tensor_to_img_op(hiding_output_op),max_outputs=1)
# tf.summary.image('hidden_noisy',tensor_to_img_op(noise_add_op),max_outputs=1)
# tf.summary.image('revealed',tensor_to_img_op(reveal_output_op),max_outputs=1)

# merged_summary_op = tf.summary.merge_all()

In [ ]:
# writer = tf.summary.FileWriter(f"logs/beta-.5-noise-large-eps",sess.graph)
# saver = tf.train.Saver()

In [ ]:
# sess.run(tf.global_variables_initializer())

In [ ]:

# total_steps = len(files_list)//BATCH_SIZE + 1
# for ep in range(3000):
    
#     for step in range(total_steps):
#         covers,secrets = get_img_batch(files_list=files_list,batch_size=BATCH_SIZE)

#         _,loss = sess.run([minimize_op,loss_op],
#                                   feed_dict={secret_tensor:secrets,cover_tensor:covers})
        
#         if step % 10 ==0 :
#             summary = sess.run(merged_summary_op,feed_dict={secret_tensor:secrets,cover_tensor:covers})
#             writer.add_summary(summary,ep*total_steps + step)
            
    
#     save_path = saver.save(sess, f"./checkpoints/beta-.5-noise-large-eps/Epoch-{ep}--Loss-{loss}.chpk")
            

In [ ]:
# save_path = saver.save(sess, f"./checkpoints/final.chpk")

# save_path = saver.save(sess, f"./checkpoints/exp-.5_noise/Epoch-{ep}--Loss-{loss}.chpk")



In [ ]:
# sess.close()

# writer.close()

In [ ]:
covers,secrets = batch
cover = covers.squeeze()
secret = secrets.squeeze()
plt.imshow(cover)
plt.show()
plt.imshow(secret)
plt.show()


In [ ]:
test_placeholder = tf.placeholder(shape=[1,224,224,3],dtype=tf.float32)
with tf.variable_scope("",reuse=True) :
    reveal_output_test = reveal_network(test_placeholder)

batch2 = get_img_batch(files_list=files_list,batch_size=1) 
covers,secrets = batch2
cover = covers.squeeze()
secret = secrets.squeeze()

# plt.imshow(cover)
# plt.show()
# plt.imshow(secret)
# plt.show()


reveal_img = sess.run([reveal_output_test],feed_dict={test_placeholder:covers})[0]
# reveal_img.shape

In [ ]:
plt.imshow(np.clip(reveal_img.squeeze(),0,1))

In [ ]:
plt.imshow(np.clip(hiding_output.squeeze(),0,1))

In [ ]:
hiding_network_output = sess.run([hiding_output_op],
                                  feed_dict={secret_tensor:secrets,cover_tensor:covers})[0]

In [ ]:
plt.imshow(np.clip(hiding_network_output[0],0,1))